# Cross-Domain Recommendation Model
* Source Domain에서 정보를 전송받아 Target Domain의 user_embeddings를 수정
* Source Domain - Restaurant
* Target Domain - Cafe / Bar
* user_embeddings 수정은 Source Domain과 Target Domain 모두에 기록을 남긴 Common User에 대해 수행
* Common User의 전체 user_embedding을 평균하는 방식으로 수정
* Common User가 아닌 경우 Target Domain의 user_embedding을 그대로 사용

In [1]:
import pandas as pd
import numpy as np
import os, random, re
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from gensim.models.doc2vec import Doc2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Embedding, Input, Flatten, Multiply, Dense, Activation, Concatenate, Dropout, BatchNormalization, Conv1D, GlobalAveragePooling1D, MaxPooling1D
pd.options.display.max_colwidth=20
pd.options.display.max_columns=999

In [2]:
rest = pd.read_csv('/content/drive/MyDrive/seminar/cdr_v2/yelp_rest_prepro.csv')
cafe = pd.read_csv('/content/drive/MyDrive/seminar/cdr_v2/yelp_cafe_prepro.csv')
bar = pd.read_csv('/content/drive/MyDrive/seminar/cdr_v2/yelp_bar_prepro.csv')

rest_doc_model = Doc2Vec.load('/content/drive/MyDrive/seminar/cdr_v2/rest_doc_model_32')
cafe_doc_model = Doc2Vec.load('/content/drive/MyDrive/seminar/cdr_v2/cafe_doc_model_32')
bar_doc_model = Doc2Vec.load('/content/drive/MyDrive/seminar/cdr_v2/bar_doc_model_32')

print(rest.shape)
print(cafe.shape)
print(bar.shape)

(1070842, 24)
(28117, 26)
(17735, 42)


In [3]:
def seed_everything(seed: int=42):
  random.seed(seed)
  np.random.seed(seed)
  os.environ['PYTHONASHSEED'] = str(seed)
  tf.random.set_seed(seed)

seed_everything(42)

In [4]:
def friends(x):
  '''
  num_friends - 친구의 수로 표현
  '''
  friends = len(x.split(','))

  return friends


def add_side_info(df=None, domain='None'):
  '''
  Rating + Review + User Profile + Context
  business_id, user_id는 knowledge transfer를 위해 지금 수정X
  '''
  if domain == 'bar':
    cols = ['business_id', 'user_id', 'stars_y', 'categories', 'Alcohol', 'BestNights', 
            'BestNights_Mon', 'BestNights_Tue', 'BestNights_Fri', 'BestNights_Wed',
            'BestNights_Thu','BestNights_Sun','BestNights_Sat', 'Music', 'video', 'dj', 
            'background_music', 'jukebox', 'no_music', 'live', 'karaoke', 'text', 'num_friends', 'fans', 'user_votes']
    new_df = df[cols]

    for col in ['num_friends', 'fans', 'user_votes', 'categories', 'Alcohol', 'BestNights', 'Music']:  # business_id, user_id
      new_df[col] = new_df[col].astype('category')
      new_df[col] = new_df[col].cat.codes.values
  
  elif domain == 'cafe':
    cols = ['business_id', 'user_id', 'stars_y', 'categories', 'OutdoorSeating', 'DriveThru', 
            'text', 'num_friends', 'fans', 'user_votes']
    new_df = df[cols]

    for col in ['num_friends', 'fans', 'user_votes', 'categories', 'OutdoorSeating', 'DriveThru']:   # business_id, user_id
      new_df[col] = new_df[col].astype('category')
      new_df[col] = new_df[col].cat.codes.values

  return new_df

def normalize(x):
  '''
  MLP / GMF / NCF - sigmoid 용 정규화 모델
  Min-Max Scaler 사용
  '''
  normalized_x = (((x-1) / (5-1)))
  return normalized_x

def de_normalize(x):
  '''
  normalize된 평점이 다시 원래 값을 갖도록 설정
  '''
  original_x = 4 * ((x - 0) / (1 - 0)) + 1
  return original_x

def rmse(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

rest['num_friends'] = rest['friends'].apply(lambda x: friends(x))
bar['num_friends'] = bar['friends'].apply(lambda x: friends(x))
cafe['num_friends'] = cafe['friends'].apply(lambda x: friends(x))

rest = rest[['business_id', 'user_id', 'stars_y', 'text', 'num_friends', 'fans', 'user_votes']]
bar = add_side_info(bar, 'bar')
cafe = add_side_info(cafe, 'cafe')

rest['stars_y_scaled'] = rest['stars_y'].apply(lambda x: normalize(x))
bar['stars_y_scaled'] = bar['stars_y'].apply(lambda x: normalize(x))
cafe['stars_y_scaled'] = cafe['stars_y'].apply(lambda x: normalize(x))

<ipython-input-4-f27b873e222d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = new_df[col].astype('category')
<ipython-input-4-f27b873e222d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = new_df[col].cat.codes.values
<ipython-input-4-f27b873e222d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [5]:
def get_embeddings(model, df=None):
  '''
  학습한 Doc2Vec 모델의 임베딩 부여
  '''
  user_embeddings = []
  for user_id in df['user_id'].unique():
    user_embedding = model.docvecs[user_id]
    user_embeddings.append(user_embedding)
  user_embeddings_dict = dict(zip(df['user_id'].unique(), user_embeddings))
  df['user_embeddings'] = df['user_id'].apply(lambda x: user_embeddings_dict[x])

  bus_embeddings = []

  for bus_id in df['business_id'].unique():
    bus_embedding = model.docvecs[bus_id]
    bus_embeddings.append(bus_embedding)
  bus_embeddings_dict = dict(zip(df['business_id'].unique(), bus_embeddings))
  df['business_embeddings'] = df['business_id'].apply(lambda x: bus_embeddings_dict[x])
  
  return df

rest_doc = get_embeddings(rest_doc_model, rest)
cafe_doc = get_embeddings(cafe_doc_model, cafe)
bar_doc = get_embeddings(bar_doc_model, bar)

<ipython-input-5-56905ebb9d88>:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding = model.docvecs[user_id]
<ipython-input-5-56905ebb9d88>:15: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  bus_embedding = model.docvecs[bus_id]
<ipython-input-5-56905ebb9d88>:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding = model.docvecs[user_id]
<ipython-input-5-56905ebb9d88>:15: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  bus_embedding = model.docvecs[bus_id]
<ipython-input-5-56905ebb9d88>:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding = model.docvecs[user_id]
<ipython-input-5-56905ebb9d88>:15: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).

## Single Cross-Domain Recommendation Model
* 단일 Source Domain에서 정보를 전송받아 Target Domain의 user_embeddings를 수정
* Source Domain, Target Domain 대상을 번갈아가며 실험 수행

In [6]:
rest_user_emb_dict = dict(zip(rest_doc['user_id'], rest_doc['user_embeddings']))
cafe_user_emb_dict = dict(zip(cafe_doc['user_id'], cafe_doc['user_embeddings']))
bar_user_emb_dict = dict(zip(bar_doc['user_id'], bar_doc['user_embeddings']))

def user_emb_mean_single(target_df, source_dict=None, target_dict=None):
  '''
  Common User에 대해 Embedding Aggregation(single cross-domain)
  Common User인 경우 user_embedding 평균
  Common User가 아닌 경우 기존 embedding 유지
  '''

  for idx, row in target_df.iterrows():
    user_id = row['user_id']

    if user_id in source_dict:
      new_user_emb = [(x + y) / 2 for x, y in zip(source_dict[user_id], target_dict[user_id])]
      target_df.at[idx, 'user_embeddings'] = new_user_emb
    
    else:
      target_df.at[idx, 'user_embeddings'] = target_dict[user_id]
  
  return target_df
  
rest_to_bar = user_emb_mean_single(bar_doc, rest_user_emb_dict, bar_user_emb_dict)
rest_to_cafe = user_emb_mean_single(cafe_doc, rest_user_emb_dict, cafe_user_emb_dict)
cafe_to_bar = user_emb_mean_single(bar_doc, cafe_user_emb_dict, bar_user_emb_dict)
bar_to_cafe = user_emb_mean_single(cafe_doc, bar_user_emb_dict, cafe_user_emb_dict)
bar_to_rest = user_emb_mean_single(rest_doc, bar_user_emb_dict, rest_user_emb_dict)
cafe_to_rest = user_emb_mean_single(rest_doc, cafe_user_emb_dict, rest_user_emb_dict)

In [7]:
# dataset making

def embeddings_making(df=None):
  '''
  inputs : 
          정보 전송이 완료된 데이터셋의 user_embeddings, business_embeddings
  
  outputs : 
          np.array() 형태의 user_embeddings, business_embeddings
  '''
  user_embeddings = []
  business_embeddings = []

  for idx in range(len(df)):
    user = df['user_embeddings'][idx]
    business = df['business_embeddings'][idx]

    user_embeddings.append(user)
    business_embeddings.append(business)
  
  return np.array(user_embeddings), np.array(business_embeddings)

def data_split(df=None, user_embeddings=None, business_embeddings=None, test_size=None):
  '''
  데이터프레임 및 np.array형태의 user/item embeddings를 
  train : valid : test = 0.6 : 0.2 : 0.2 크기로 분할
  '''
  train, test = train_test_split(df, test_size=test_size, random_state=42)
  train_user, test_user = train_test_split(user_embeddings, test_size=test_size, random_state=42)
  train_bus, test_bus = train_test_split(business_embeddings, test_size=test_size, random_state=42)

  return train, test, train_user, test_user, train_bus, test_bus

rb_user_emb, rb_bus_emb = embeddings_making(rest_to_bar)
rb_train, rb_test, rb_train_user, rb_test_user, rb_train_bus, rb_test_bus = data_split(rest_to_bar, rb_user_emb, rb_bus_emb, 0.2)
rb_train, rb_valid, rb_train_user, rb_valid_user, rb_train_bus, rb_valid_bus = data_split(rb_train, rb_train_user, rb_train_bus, 0.25)

rc_user_emb, rc_bus_emb = embeddings_making(rest_to_cafe)
rc_train, rc_test, rc_train_user, rc_test_user, rc_train_bus, rc_test_bus = data_split(rest_to_cafe, rc_user_emb, rc_bus_emb, 0.2)
rc_train, rc_valid, rc_train_user, rc_valid_user, rc_train_bus, rc_valid_bus = data_split(rc_train, rc_train_user, rc_train_bus, 0.25)

cb_user_emb, cb_bus_emb = embeddings_making(cafe_to_bar)
cb_train, cb_test, cb_train_user, cb_test_user, cb_train_bus, cb_test_bus = data_split(cafe_to_bar, cb_user_emb, cb_bus_emb, 0.2)
cb_train, cb_valid, cb_train_user, cb_valid_user, cb_train_bus, cb_valid_bus = data_split(cb_train, cb_train_user, cb_train_bus, 0.25)

bc_user_emb, bc_bus_emb = embeddings_making(bar_to_cafe)
bc_train, bc_test, bc_train_user, bc_test_user, bc_train_bus, bc_test_bus = data_split(bar_to_cafe, bc_user_emb, bc_bus_emb, 0.2)
bc_train, bc_valid, bc_train_user, bc_valid_user, bc_train_bus, bc_valid_bus = data_split(bc_train, bc_train_user, bc_train_bus, 0.25)

br_user_emb, br_bus_emb = embeddings_making(bar_to_rest)
br_train, br_test, br_train_user, br_test_user, br_train_bus, br_test_bus = data_split(bar_to_rest, br_user_emb, br_bus_emb, 0.2)
br_train, br_valid, br_train_user, br_valid_user, br_train_bus, br_valid_bus = data_split(br_train, br_train_user, br_train_bus, 0.25)

cr_user_emb, cr_bus_emb = embeddings_making(cafe_to_rest)
cr_train, cr_test, cr_train_user, cr_test_user, cr_train_bus, cr_test_bus = data_split(cafe_to_rest, cr_user_emb, cr_bus_emb, 0.2)
cr_train, cr_valid, cr_train_user, cr_valid_user, cr_train_bus, cr_valid_bus = data_split(cr_train, cr_train_user, cr_train_bus, 0.25)

In [84]:
# Proposed Model
cdr_configs = {'vector_length' : 32, 'user_item_embed' : 32, 'drop_rate' : 0.2, 
           'dense_1' : 256, 'dense_2' : 128, 'dense_3' : 32, 'dense_4' : 8, 'output' : 1}

class CDR(tf.keras.Model):

  def __init__(self, **cdr_configs):
    super(CDR, self).__init__(name='CDR')
    
    user_input = Input(shape=(cdr_configs['vector_length'], ), name='user_input')
    business_input = Input(shape=(cdr_configs['vector_length']), name='business_input')

    user_emb = Dense(cdr_configs['user_item_embed'], activation='relu', name='user_emb')(user_input)
    business_emb = Dense(cdr_configs['user_item_embed'], activation='relu', name='business_emb')(business_input)
    user_emb = Flatten(name='user_flat')(user_emb)
    business_emb = Flatten(name='business_flat')(business_emb)
    concat = Concatenate(name='concat')([user_emb, business_emb])
    drop_1 = Dropout(rate=cdr_configs['drop_rate'], name='drop_1')(concat)
    
    mlp_1 = Dense(cdr_configs['dense_1'], activation='relu', name='mlp_1')(drop_1)
    drop_2 = Dropout(rate=cdr_configs['drop_rate'], name='drop_2')(mlp_1)
    bn_1 = BatchNormalization(name='bn_1')(drop_2)

    mlp_2 = Dense(cdr_configs['dense_2'], activation='relu', name='mlp_2')(bn_1)
    drop_3 = Dropout(rate=cdr_configs['drop_rate'], name='drop_3')(mlp_2)
    bn_2 = BatchNormalization(name='bn_2')(drop_3)

    mlp_3 = Dense(cdr_configs['dense_3'], activation='relu', name='mlp_3')(bn_2)
    mlp_4 = Dense(cdr_configs['dense_4'], activation='relu', name='mlp_4')(mlp_3)
    output = Dense(cdr_configs['output'], activation='sigmoid', name='output')(mlp_4)

    self.model = Model([user_input, business_input], output, name='CDR')
  
  def get_model(self):
    model = self.model
    return model

es = EarlyStopping(monitor='val_loss', mode = 'min', patience=5, min_delta=0.001, restore_best_weights = True)
rp = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=2, min_delta=0.001)

In [85]:
# Restaurant -> Bar Fitting
rest_to_bar_cdr = CDR(**cdr_configs).get_model()
print(rest_to_bar_cdr.summary())
rest_to_bar_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = rest_to_bar_cdr.fit(x=[rb_train_user, rb_train_bus],
                        y=rb_train['stars_y_scaled'], validation_data=([rb_valid_user, rb_valid_bus], rb_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [64]:
# Restaurant -> Bar Prediction
pred_mlp = rest_to_bar_cdr.predict([rb_test_user, rb_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(rb_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(rb_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

111/111 [==============================] - 0s 2ms/step
rmse : 0.9564603366574758
mae : 0.7571333717688191


In [86]:
# Restaurant -> Cafe Fitting
rest_to_cafe_cdr = CDR(**cdr_configs).get_model()
print(rest_to_cafe_cdr.summary())
rest_to_cafe_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = rest_to_cafe_cdr.fit(x=[rc_train_user, rc_train_bus],
                        y=rc_train['stars_y_scaled'], validation_data=([rc_valid_user, rc_valid_bus], rc_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [87]:
# Restaurant -> Cafe Prediction
pred_mlp = rest_to_cafe_cdr.predict([rc_test_user, rc_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(rc_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(rc_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

176/176 [==============================] - 0s 2ms/step
rmse : 0.9141813975413384
mae : 0.7152541452968443


In [88]:
rmse = [1.0203324539143392, 0.9509480104735634, 0.932650756945359, 0.9264261843623551, 0.9141813975413384]
mae = [0.8185259874480887, 0.7558637652678646, 0.7350760151447985, 0.724873682831124, 0.7152541452968443]
print(np.mean(rmse))
print(np.mean(mae))

0.9489077606473911
0.749918719197744


In [ ]:
# Cafe -> Bar Fitting
cafe_to_bar_cdr = CDR(**cdr_configs).get_model()
print(cafe_to_bar_cdr.summary())
cafe_to_bar_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = cafe_to_bar_cdr.fit(x=[cb_train_user, cb_train_bus],
                        y=cb_train['stars_y_scaled'], validation_data=([cb_valid_user, cb_valid_bus], cb_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Cafe -> Bar Prediction
pred_mlp = cafe_to_bar_cdr.predict([cb_test_user, cb_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(cb_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(cb_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

111/111 [==============================] - 0s 2ms/step
rmse : 1.1282247484261845
mae : 0.916542917733667


In [ ]:
rmse = [1.0123167143585878, 1.0510185013779367, 1.2001189258480245, 0.9542414375094511, 1.1282247484261845]
mae = [0.8122459155994971, 0.851650158284179, 0.9799637421104381, 0.7616264924285174, 0.916542917733667]

print(np.mean(rmse))
print(np.mean(mae))

1.069184065504037
0.8644058452312595


In [ ]:
# Bar -> Cafe Fitting
bar_to_cafe_cdr = CDR(**cdr_configs).get_model()
print(bar_to_cafe_cdr.summary())
bar_to_cafe_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = cafe_to_bar_cdr.fit(x=[bc_train_user, bc_train_bus],
                        y=bc_train['stars_y_scaled'], validation_data=([bc_valid_user, bc_valid_bus], bc_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Bar -> Cafe Prediction
pred_mlp = bar_to_cafe_cdr.predict([bc_test_user, bc_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(bc_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(bc_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

176/176 [==============================] - 0s 1ms/step
rmse : 1.4733267687023681
mae : 1.28096471887988


In [ ]:
# Bar -> Rest Fitting
bar_to_rest_cdr = CDR(**cdr_configs).get_model()
print(bar_to_rest_cdr.summary())
bar_to_rest_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = bar_to_rest_cdr.fit(x=[br_train_user, br_train_bus],
                        y=br_train['stars_y_scaled'], validation_data=([br_valid_user, br_valid_bus], br_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Bar -> Rest Prediction
pred_mlp = bar_to_rest_cdr.predict([br_test_user, br_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(br_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(br_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

6693/6693 [==============================] - 11s 2ms/step
rmse : 1.086288002939854
mae : 0.8617794333793879


In [ ]:
# Cafe -> Rest Fitting
cafe_to_rest_cdr = CDR(**cdr_configs).get_model()
print(cafe_to_rest_cdr.summary())
cafe_to_rest_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = cafe_to_rest_cdr.fit(x=[cr_train_user, cr_train_bus],
                        y=cr_train['stars_y_scaled'], validation_data=([cr_valid_user, cr_valid_bus], cr_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Cafe -> Rest Prediction
pred_mlp = cafe_to_rest_cdr.predict([br_test_user, br_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(cr_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(cr_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

6693/6693 [==============================] - 12s 2ms/step
rmse : 1.0837316763933214
mae : 0.858464809411161


## Multi Cross-Domain Recommendation Model
* 2개의 Source Domain에서 정보를 전송받아 Target Domain의 user_embeddings를 수정
* 3개 도메인에 모두 기록을 남긴 User와 2개 도메인에 기록을 남긴 User를 구분
* Source Domain, Target Domain 대상을 번갈아가며 실험 수행

In [ ]:
rest_user_emb_dict = dict(zip(rest_doc['user_id'], rest_doc['user_embeddings']))
cafe_user_emb_dict = dict(zip(cafe_doc['user_id'], cafe_doc['user_embeddings']))
bar_user_emb_dict = dict(zip(bar_doc['user_id'], bar_doc['user_embeddings']))

def user_emb_mean_multi(target_df, source_dict_1=None, source_dict_2=None, target_dict=None):
  '''
  Common User에 대해 Embedding Aggregation(multi cross-domain)
  1) 3개의 데이터셋 모두에 Common User인 경우 user_embedding: 3 domain's user embedding sum / 3
  2) 1개의 source domain과 target domain에 Common User인 경우 user_embedding : 2 domain's user embedding sum / 2
  
  1) 2)가 아닌 User의 경우 기존 embedding 유지
  '''

  for idx, row in target_df.iterrows():
    user_id = row['user_id']

    if user_id in source_dict_1 and user_id in source_dict_2 and user_id in target_dict:
      new_user_emb = [(x + y + z) / 3 for x, y, z in zip(source_dict_1[user_id], source_dict_2[user_id], target_dict[user_id])]
      target_df.at[idx, 'user_embeddings'] = new_user_emb
    
    elif user_id in source_dict_1 and user_id in target_dict:
      new_user_emb = [(x + y) / 2 for x, y in zip(source_dict_1[user_id], target_dict[user_id])]
      target_df.at[idx, 'user_embeddings'] = new_user_emb
    
    elif user_id in source_dict_2 and user_id in target_dict:
      new_user_emb = [(x + y) / 2 for x, y in zip(source_dict_2[user_id], target_dict[user_id])]
    
    else:
      target_df.at[idx, 'user_embeddings'] = target_dict[user_id]
  
  return target_df

#rest_cafe_to_bar = user_emb_mean_multi(bar_doc, rest_user_emb_dict, cafe_user_emb_dict, bar_user_emb_dict)
#rest_bar_to_cafe = user_emb_mean_multi(cafe_doc, rest_user_emb_dict, bar_user_emb_dict, cafe_user_emb_dict)
cafe_bar_to_rest = user_emb_mean_multi(rest_doc, cafe_user_emb_dict, bar_user_emb_dict, rest_user_emb_dict)

In [ ]:
# dataset making

def embeddings_making(df=None):
  '''
  inputs : 
          정보 전송이 완료된 데이터셋의 user_embeddings, business_embeddings
  
  outputs : 
          np.array() 형태의 user_embeddings, business_embeddings
  '''
  user_embeddings = []
  business_embeddings = []

  for idx in range(len(df)):
    user = df['user_embeddings'][idx]
    business = df['business_embeddings'][idx]

    user_embeddings.append(user)
    business_embeddings.append(business)
  
  return np.array(user_embeddings), np.array(business_embeddings)

def data_split(df=None, user_embeddings=None, business_embeddings=None, test_size=None):
  '''
  데이터프레임 및 np.array형태의 user/item embeddings를 
  train : valid : test = 0.6 : 0.2 : 0.2 크기로 분할
  '''
  train, test = train_test_split(df, test_size=test_size, random_state=42)
  train_user, test_user = train_test_split(user_embeddings, test_size=test_size, random_state=42)
  train_bus, test_bus = train_test_split(business_embeddings, test_size=test_size, random_state=42)

  return train, test, train_user, test_user, train_bus, test_bus

# rcb_user_emb, rcb_bus_emb = embeddings_making(rest_cafe_to_bar)
# rcb_train, rcb_test, rcb_train_user, rcb_test_user, rcb_train_bus, rcb_test_bus = data_split(rest_cafe_to_bar, rcb_user_emb, rcb_bus_emb, 0.2)
# rcb_train, rcb_valid, rcb_train_user, rcb_valid_user, rcb_train_bus, rcb_valid_bus = data_split(rcb_train, rcb_train_user, rcb_train_bus, 0.25)
# rbc_user_emb, rbc_bus_emb = embeddings_making(rest_bar_to_cafe)
# rbc_train, rbc_test, rbc_train_user, rbc_test_user, rbc_train_bus, rbc_test_bus = data_split(rest_bar_to_cafe, rbc_user_emb, rbc_bus_emb, 0.2)
# rbc_train, rbc_valid, rbc_train_user, rbc_valid_user, rbc_train_bus, rbc_valid_bus = data_split(rbc_train, rbc_train_user, rbc_train_bus, 0.25)
cbr_user_emb, cbr_bus_emb = embeddings_making(cafe_bar_to_rest)
cbr_train, cbr_test, cbr_train_user, cbr_test_user, cbr_train_bus, cbr_test_bus = data_split(cafe_bar_to_rest, cbr_user_emb, cbr_bus_emb, 0.2)
cbr_train, cbr_valid, cbr_train_user, cbr_valid_user, cbr_train_bus, cbr_valid_bus = data_split(cbr_train, cbr_train_user, cbr_train_bus, 0.25)

In [ ]:
# Restaurant & Cafe -> Bar Fitting
rest_cafe_to_bar_cdr = CDR(**cdr_configs).get_model()
print(rest_cafe_to_bar_cdr.summary())
rest_cafe_to_bar_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = rest_cafe_to_bar_cdr.fit(x=[rcb_train_user, rcb_train_bus],
                        y=rcb_train['stars_y_scaled'], validation_data=([rcb_valid_user, rcb_valid_bus], rcb_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Restaurant & Cafe -> Bar Prediction
pred_mlp = rest_cafe_to_bar_cdr.predict([rcb_test_user, rcb_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(rcb_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(rcb_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

111/111 [==============================] - 0s 2ms/step
rmse : 1.096279243858441
mae : 0.884777257769821


In [ ]:
# Restaurant & Bar -> Cafe Fitting
rest_bar_to_cafe_cdr = CDR(**cdr_configs).get_model()
print(rest_bar_to_cafe_cdr.summary())
rest_bar_to_cafe_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = rest_bar_to_cafe_cdr.fit(x=[rbc_train_user, rbc_train_bus],
                        y=rbc_train['stars_y_scaled'], validation_data=([rbc_valid_user, rbc_valid_bus], rbc_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Restaurant & Bar -> Cafe Prediction
pred_mlp = rest_bar_to_cafe_cdr.predict([rbc_test_user, rbc_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(rbc_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(rbc_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

176/176 [==============================] - 0s 2ms/step
rmse : 0.9315531367850356
mae : 0.7380769234336118


In [ ]:
# Cafe & Bar -> Rest Fitting
cafe_bar_to_rest_cdr = CDR(**cdr_configs).get_model()
print(cafe_bar_to_rest_cdr.summary())
cafe_bar_to_rest_cdr.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
result = cafe_bar_to_rest_cdr.fit(x=[cbr_train_user, cbr_train_bus],
                        y=cbr_train['stars_y_scaled'], validation_data=([cbr_valid_user, cbr_valid_bus], cbr_valid['stars_y_scaled']), epochs=100, batch_size=512, callbacks=[es, rp])

Model: "CDR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 32)]         0           []                               
                                                                                                  
 business_input (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 user_emb (Dense)               (None, 32)           1056        ['user_input[0][0]']             
                                                                                                  
 business_emb (Dense)           (None, 32)           1056        ['business_input[0][0]']         
                                                                                                

In [ ]:
# Cafe & Bar -> Rest Prediction
pred_mlp = cafe_bar_to_rest_cdr.predict([cbr_test_user, cbr_test_bus])
pred_mlp = 4 *((pred_mlp - pred_mlp.min())/(pred_mlp.max() - pred_mlp.min())) + 1  # min-max scaler
rmse_temp = mean_squared_error(cbr_test['stars_y'], pred_mlp, squared=False)
mae_temp = mean_absolute_error(cbr_test['stars_y'], pred_mlp)
print(f'rmse : {rmse_temp}')
print(f'mae : {mae_temp}')

6693/6693 [==============================] - 12s 2ms/step
rmse : 1.086148799841927
mae : 0.8596886050562044


In [ ]:
rmse = [1.08752905214556, 1.084979216852343, 1.0869459541224147, 1.083243485750138, 1.086148799841927]
mae = [0.8630214407644503, 0.8610160157654785, 0.8617105823643023, 0.855889611510619, 0.8596886050562044]

print(np.mean(rmse))
print(np.mean(mae))

1.0857693017424765
0.8602652510922109
